# **QUESTION 1**

In [1]:
################################### Question 1 (Preparation and Data Import) ###################################

import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy.linalg import inv
from scipy.optimize import minimize
from scipy.stats import chi2
from scipy.stats import norm
import pyblp


np.random.seed(42)

df1 = pd.read_csv("Sales_Hausman.csv",sep=",")
df2 = pd.read_csv("OTC_Demographics.csv",sep="\t")
df3 = pd.read_csv("OTC_Instruments.csv",sep="\t")


df1.head()


,store,week,brand,sales_,count,price_,prom_,cost_,Hausman
0,2,1,1,16,14181,3.29,0.0,2.06,3.300641
1,2,2,1,12,13965,3.27,0.0,2.04,3.298750
2,2,3,1,6,13538,3.37,0.0,2.15,3.311178
3,2,4,1,12,13735,3.30,0.0,2.07,3.308676
4,2,5,1,10,13735,3.34,0.0,2.12,3.298289


In [2]:
################################### Summary statistics for the table 1 ###################################

# Calculate the total sales
total_sales = df1['sales_'].sum()

# Calculate the sales share for each brand
brand_sales = df1.groupby('brand')['sales_'].sum()


# Generate share variables for each brand
share_1 = brand_sales[1] / total_sales
share_2 = brand_sales[2] / total_sales
share_3 = brand_sales[3] / total_sales
share_4 = brand_sales[4] / total_sales
share_5 = brand_sales[5] / total_sales
share_6 = brand_sales[6] / total_sales
share_7 = brand_sales[7] / total_sales
share_8 = brand_sales[8] / total_sales
share_9 = brand_sales[9] / total_sales
share_10 = brand_sales[10] / total_sales
share_11 = brand_sales[11] / total_sales

# Print the share variables
print(f"share_1: {share_1}")
print(f"share_2: {share_2}")
print(f"share_3: {share_3}")
print(f"share_4: {share_4}")
print(f"share_5: {share_5}")
print(f"share_6: {share_6}")
print(f"share_7: {share_7}")
print(f"share_8: {share_8}")
print(f"share_9: {share_9}")
print(f"share_10: {share_10}")
print(f"share_11: {share_11}")


share_1: 0.14513692602512893
share_2: 0.17988041419128795
share_3: 0.11786960705561754
share_4: 0.11845629580078901
share_5: 0.07822151064127274
share_6: 0.035480961401911947
share_7: 0.04076390164464756
share_8: 0.03445288532976568
share_9: 0.0800747891073284
share_10: 0.09615938194808078
share_11: 0.07350332685416947


In [3]:
######################################### Weighted average price and cost for each brand #########################################

for i in range(1, 12):
    weighted_avg_price_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['price_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_price_{i}: {weighted_avg_price_i}")

# Calculate the weighted average cost (wholesale price) for each brand

for i in range(1, 12):
    weighted_avg_cost_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['cost_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_cost_{i}: {weighted_avg_cost_i}")


weighted_avg_price_1: 3.414884775217227
weighted_avg_price_2: 4.888457470318381
weighted_avg_price_3: 6.935243987533144
weighted_avg_price_4: 2.938778235511942
weighted_avg_price_5: 5.02520433197813
weighted_avg_price_6: 8.071122701282645
weighted_avg_price_7: 2.670661107001143
weighted_avg_price_8: 3.600319089679319
weighted_avg_price_9: 3.9389793892084364
weighted_avg_price_10: 1.8611150392017106
weighted_avg_price_11: 4.29954384394465
weighted_avg_cost_1: 2.17952871174915
weighted_avg_cost_2: 3.6696483928489783
weighted_avg_cost_3: 5.748667255896171
weighted_avg_cost_4: 2.0292830031475653
weighted_avg_cost_5: 3.6047360857984017
weighted_avg_cost_6: 6.104660794313088
weighted_avg_cost_7: 1.8421487658887619
weighted_avg_cost_8: 2.484719503461447
weighted_avg_cost_9: 3.7093806491372225
weighted_avg_cost_10: 0.9077034925160371
weighted_avg_cost_11: 1.8716247062772742


# **QUESTION 2**


In [4]:
####################################### Question 2 (Data Preparation) #######################################

df1['sales_per_count'] = df1['sales_'] / df1['count']

df1['share_0'] = 1 - df1.groupby(['store', 'week'])['sales_per_count'].transform('sum')

df1['Y'] = np.log(df1['sales_per_count']) - np.log(df1['share_0'])

df1['market_id'] = df1['store'].astype(str) + '_' + df1['week'].astype(str)
df1.head()

df1.head()

,store,week,brand,sales_,count,price_,prom_,cost_,Hausman,sales_per_count,share_0,Y,market_id
0,2,1,1,16,14181,3.29,0.0,2.06,3.300641,0.001128,0.993724,-6.780774,2_1
1,2,2,1,12,13965,3.27,0.0,2.04,3.298750,0.000859,0.993913,-7.053298,2_2
2,2,3,1,6,13538,3.37,0.0,2.15,3.311178,0.000443,0.995199,-7.716683,2_3
3,2,4,1,12,13735,3.30,0.0,2.07,3.308676,0.000874,0.993957,-7.036735,2_4
4,2,5,1,10,13735,3.34,0.0,2.12,3.298289,0.000728,0.995268,-7.220374,2_5


# **QUESTION 2.A**

In [5]:
##################################### OLS Q2.(a) #####################################

# Define the independent variables
X = df1[['prom_', 'price_']]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     315.2
Date:                Tue, 11 Feb 2025   Prob (F-statistic):          1.72e-136
Time:                        05:02:19   Log-Likelihood:                -50711.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                         1.015e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.7660      0.012   -627.345      0.0

# **QUESTION 2.B**

In [6]:
######################################## Generate Dummies for Product Type ########################################

# Create dummy variables
df1['tyl'] = df1['brand'].isin([1, 2, 3]).astype(int)
df1['adv'] = df1['brand'].isin([4, 5, 6]).astype(int)
df1['bay'] = df1['brand'].isin([7, 8, 9]).astype(int)
df1['sto'] = df1['brand'].isin([10, 11]).astype(int)

# Display the first few rows to verify
df1.head()

# Create separate dummy variables for each brand
for brand in df1['brand'].unique():
    df1[f'brand_{brand}'] = (df1['brand'] == brand).astype(int)

# Display the first few rows to verify
df1.head()

,store,week,brand,sales_,count,price_,prom_,cost_,Hausman,sales_per_count,...,brand_2,brand_3,brand_4,brand_5,brand_6,brand_7,brand_8,brand_9,brand_10,brand_11
0,2,1,1,16,14181,3.29,0.0,2.06,3.300641,0.001128,...,0,0,0,0,0,0,0,0,0,0
1,2,2,1,12,13965,3.27,0.0,2.04,3.298750,0.000859,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1,6,13538,3.37,0.0,2.15,3.311178,0.000443,...,0,0,0,0,0,0,0,0,0,0
3,2,4,1,12,13735,3.30,0.0,2.07,3.308676,0.000874,...,0,0,0,0,0,0,0,0,0,0
4,2,5,1,10,13735,3.34,0.0,2.12,3.298289,0.000728,...,0,0,0,0,0,0,0,0,0,0


In [7]:
##################################### OLS Q2.(b) #####################################

# Define the independent variables including the dummy variables
X = df1[['price_', 'prom_', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']] 


# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model_with_dummies = sm.OLS(Y, X).fit()

# Print the regression results
print(model_with_dummies.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.460
Method:                 Least Squares   F-statistic:                     2733.
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        05:02:19   Log-Likelihood:                -39156.
No. Observations:               38544   AIC:                         7.834e+04
Df Residuals:                   38531   BIC:                         7.845e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.9639      0.041   -145.411      0.0

# **QUESTION 2.C**

In [8]:
############################################### Fixed Effects for Everything ###############################################

# Generate new variables for each unique combination of store number and tyl, adv, bay, and sto
for store in df1['store'].unique():
    df1[f'tyl_store_{store}'] = df1['tyl'] * (df1['store'] == store).astype(int)
    df1[f'adv_store_{store}'] = df1['adv'] * (df1['store'] == store).astype(int)
    df1[f'bay_store_{store}'] = df1['bay'] * (df1['store'] == store).astype(int)
    df1[f'sto_store_{store}'] = df1['sto'] * (df1['store'] == store).astype(int)

# Display the first few rows to verify
df1.head()


/tmp/ipykernel_20075/3716583663.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'sto_store_{store}'] = df1['sto'] * (df1['store'] == store).astype(int)
/tmp/ipykernel_20075/3716583663.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'tyl_store_{store}'] = df1['tyl'] * (df1['store'] == store).astype(int)
/tmp/ipykernel_20075/3716583663.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

,store,week,brand,sales_,count,price_,prom_,cost_,Hausman,sales_per_count,...,bay_store_121,sto_store_121,tyl_store_122,adv_store_122,bay_store_122,sto_store_122,tyl_store_123,adv_store_123,bay_store_123,sto_store_123
0,2,1,1,16,14181,3.29,0.0,2.06,3.300641,0.001128,...,0,0,0,0,0,0,0,0,0,0
1,2,2,1,12,13965,3.27,0.0,2.04,3.298750,0.000859,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1,6,13538,3.37,0.0,2.15,3.311178,0.000443,...,0,0,0,0,0,0,0,0,0,0
3,2,4,1,12,13735,3.30,0.0,2.07,3.308676,0.000874,...,0,0,0,0,0,0,0,0,0,0
4,2,5,1,10,13735,3.34,0.0,2.12,3.298289,0.000728,...,0,0,0,0,0,0,0,0,0,0


In [23]:
##################################### OLS Q2.(c) Fixed Effects Model #####################################

# Define the independent variables including the dummy variables
X_fixed_effects = df1[['price_', 'prom_'] + [col for col in df1.columns if '_store' in col]]


# Add a constant to the independent variables
X_fixed_effects = sm.add_constant(X_fixed_effects)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
fixed_effects_model = sm.OLS(Y, X_fixed_effects).fit()


# Print the regression results
print(fixed_effects_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     105.9
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        05:04:00   Log-Likelihood:                -39578.
No. Observations:               38544   AIC:                         7.974e+04
Df Residuals:                   38250   BIC:                         8.226e+04
Df Model:                         293                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -7.2002      0.010   -689.905

# **QUESTION 2.D**

In [10]:
####################################### OLS Q2.(d) #####################################

# Define the independent variables and the instrument
X = df1[['prom_']]
X = sm.add_constant(X)

instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X['price_hat'] = df1['price_hat']
second_stage = sm.OLS(Y, X).fit()

# Print the regression results
print(second_stage.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     117.7
Date:                Tue, 11 Feb 2025   Prob (F-statistic):           1.09e-51
Time:                        05:02:21   Log-Likelihood:                -50906.
No. Observations:               38544   AIC:                         1.018e+05
Df Residuals:                   38541   BIC:                         1.018e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.9482      0.013   -611.462      0.0

/tmp/ipykernel_20075/3327803042.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1['price_hat'] = first_stage.fittedvalues


In [31]:
##################################### IV with Dummies #####################################

# Define the independent variables and the instrument
X = df1[['prom_', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']]
X = sm.add_constant(X)

instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.predict()

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X['price_hat'] = df1['price_hat']
second_stage = sm.OLS(Y, X).fit()

# Print the regression results
print(second_stage.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     2561.
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        05:25:20   Log-Likelihood:                -39722.
No. Observations:               38544   AIC:                         7.947e+04
Df Residuals:                   38531   BIC:                         7.958e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.3188      0.066   -111.413      0.0

In [12]:
##################################### IV with Fixed Effects #####################################

# Define the independent variables and the instrument
X_fixed_effects = df1[['prom_'] + [col for col in df1.columns if '_store' in col]]
X_fixed_effects = sm.add_constant(X_fixed_effects)
instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X_fixed_effects['price_hat'] = df1['price_hat']
model_iv_fixed_effects = sm.OLS(Y, X_fixed_effects).fit()

# Print the regression results
print(model_iv_fixed_effects.summary())


#print t statistic (-57.541554716524786) 
print(model_iv_fixed_effects.tvalues['price_hat'])
#print confidence interval (-0.158776 -0.148316) for "price_" coefficient
print(model_iv_fixed_effects.conf_int(alpha=0.05, cols=None))

# Print the coefficient for price
print(f"Price coefficient: {model_iv_fixed_effects.params['price_hat']}")



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     87.28
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        05:02:25   Log-Likelihood:                -41156.
No. Observations:               38544   AIC:                         8.290e+04
Df Residuals:                   38250   BIC:                         8.542e+04
Df Model:                         293                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -7.3622      0.012   -620.879

# **QUESTION 2.E**

In [13]:
################################################# Code used to generate Hausman Variable #################################################

# Define a function to calculate the Hausman variable
#def calculate_hausman(row):
    #other_stores = df1[(df1['store'] != row['store']) & (df1['week'] == row['week']) & (df1['brand'] == row['brand'])]
    #if not other_stores.empty:
    #    weighted_avg_price_other_stores = (other_stores['sales_'] * other_stores['price_']).sum() / other_stores['sales_'].sum()
    #else:
    #   weighted_avg_price_other_stores = np.nan
    #return weighted_avg_price_other_stores

# Apply the function to each row in the dataframe
#['Hausman'] = df1.apply(calculate_hausman, axis=1)

# Display the first few rows to verify
#df1.head()

# Add the Hausman variable to the original dataframe

#dfHausman = pd.read_csv("OTC_Sales.csv",sep="\t")
#dfHausman['Hausman'] = df1['Hausman']

# Save the updated dataframe to a new CSV file
#dfHausman.to_csv("Sales_Hausman.csv", index=False)

In [14]:
# First stage: regress price_ on the instrument Hausman
first_stage_hausman = sm.OLS(df1['price_'], sm.add_constant(df1['Hausman'])).fit()
df1['price_hat_hausman'] = first_stage_hausman.fittedvalues

# Second stage: regress Y on the predicted values of price_ and prom_
X_hausman = df1[['prom_']]
X_hausman = sm.add_constant(X_hausman)
X_hausman['price_hat_hausman'] = df1['price_hat_hausman']
second_stage_hausman = sm.OLS(Y, X_hausman).fit()

# Print the regression results
print(second_stage_hausman.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     316.8
Date:                Tue, 11 Feb 2025   Prob (F-statistic):          3.57e-137
Time:                        05:02:25   Log-Likelihood:                -50709.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                         1.015e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -7.7636      0.01

/tmp/ipykernel_20075/3992901645.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1['price_hat_hausman'] = first_stage_hausman.fittedvalues


In [32]:
# Define the independent variables including the dummy variables
X_dummies_hausman = df1[['prom_', 'price_hat_hausman', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']]

# Add a constant to the independent variables
X_dummies_hausman = sm.add_constant(X_dummies_hausman)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
second_stage_dummies_hausman = sm.OLS(Y, X_dummies_hausman).fit()

# Print the regression results
print(second_stage_dummies_hausman.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     2826.
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        06:13:41   Log-Likelihood:                -38856.
No. Observations:               38544   AIC:                         7.774e+04
Df Residuals:                   38531   BIC:                         7.785e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -5.2702      0.05

In [16]:
# Define the independent variables and the instrument
X_fixed_effects_hausman = df1[['prom_'] + [col for col in df1.columns if '_store' in col]]
X_fixed_effects_hausman = sm.add_constant(X_fixed_effects_hausman)
instrument_hausman = df1['Hausman']

# First stage: regress price_ on the instrument Hausman and other exogenous variables
first_stage_hausman = sm.OLS(df1['price_'], sm.add_constant(instrument_hausman)).fit()
df1['price_hat_hausman'] = first_stage_hausman.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X_fixed_effects_hausman['price_hat_hausman'] = df1['price_hat_hausman']
model_iv_fixed_effects_hausman = sm.OLS(Y, X_fixed_effects_hausman).fit()

# Print the regression results
print(model_iv_fixed_effects_hausman.summary())

# Print the coefficient for price
print(f"Price coefficient: {model_iv_fixed_effects_hausman.params['price_hat_hausman']}")

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     107.0
Date:                Tue, 11 Feb 2025   Prob (F-statistic):               0.00
Time:                        05:02:27   Log-Likelihood:                -39488.
No. Observations:               38544   AIC:                         7.956e+04
Df Residuals:                   38250   BIC:                         8.208e+04
Df Model:                         293                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -7.1829      0.01

# **QUESTION 2.G**

In [34]:
# Define the coefficient of price from model (a)
price_coefficient = -0.0514

# Calculate the elasticity
df1['elasticity'] = price_coefficient * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl = (df1['sales_'] * df1['elasticity'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv = (df1['sales_'] * df1['elasticity'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay = (df1['sales_'] * df1['elasticity'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto = (df1['sales_'] * df1['elasticity'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl: {elasticity_tyl}")
print(f"Elasticity for adv: {elasticity_adv}")
print(f"Elasticity for bay: {elasticity_bay}")
print(f"Elasticity for sto: {elasticity_sto}")


Elasticity for tyl: -0.11258705271206843
Elasticity for adv: -0.05278634561724045
Elasticity for bay: -0.02817155809285109
Elasticity for sto: -0.025423809771131196


In [18]:
# Define the coefficient of price from model (b)
price_coefficient_b = model_with_dummies.params['price_']
print(price_coefficient_b)
# Calculate the elasticity
df1['elasticity_b'] = price_coefficient_b * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl_b = (df1['sales_'] * df1['elasticity_b'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv_b = (df1['sales_'] * df1['elasticity_b'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay_b = (df1['sales_'] * df1['elasticity_b'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto_b = (df1['sales_'] * df1['elasticity_b'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl (part b): {elasticity_tyl_b}")
print(f"Elasticity for adv (part b): {elasticity_adv_b}")
print(f"Elasticity for bay (part b): {elasticity_bay_b}")
print(f"Elasticity for sto (part b): {elasticity_sto_b}")

-0.34161104979828033
Elasticity for tyl (part b): -0.7482681181063039
Elasticity for adv (part b): -0.35082488212685536
Elasticity for bay (part b): -0.18723181973836772
Elasticity for sto (part b): -0.1689699289063795


/tmp/ipykernel_20075/2847781573.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1['elasticity_b'] = price_coefficient_b * df1['price_'] * (1 - df1['sales_per_count'])


In [19]:
# Define the coefficient of price from model (c)
price_coefficient_c = fixed_effects_model.params['price_']

print(price_coefficient_c)

# Calculate the elasticity
df1['elasticity_c'] = price_coefficient_c * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl_c = (df1['sales_'] * df1['elasticity_c'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv_c = (df1['sales_'] * df1['elasticity_c'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay_c = (df1['sales_'] * df1['elasticity_c'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto_c = (df1['sales_'] * df1['elasticity_c'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl (part c): {elasticity_tyl_c}")
print(f"Elasticity for adv (part c): {elasticity_adv_c}")
print(f"Elasticity for bay (part c): {elasticity_bay_c}")
print(f"Elasticity for sto (part c): {elasticity_sto_c}")

-0.188034616488536
Elasticity for tyl (part c): -0.4118728264258438
Elasticity for adv (part c): -0.19310623062196738
Elasticity for bay (part c): -0.10305891287692155
Elasticity for sto (part c): -0.0930069322955671


/tmp/ipykernel_20075/388466451.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1['elasticity_c'] = price_coefficient_c * df1['price_'] * (1 - df1['sales_per_count'])


# **QUESTION 3**

In [20]:
######################################### Data Preparation for Question 3 #########################################

# Generate a single dummy variable for brands 1 to 9
df1['brand_'] = df1['brand'].isin(range(1, 10)).astype(int)

df_merged = pd.merge(df1, df2, on=['store'], how='left')
df_merged = pd.merge(df_merged, df3, on=['store','week','brand'], how='left')

df_merged['cost_']=df_merged['cost__x'].fillna(0)

df_merged.drop(columns=['cost__x'], inplace=True)

# Generate a value randomly drawn from N(0,1) for each observation
df_merged['V'] = np.random.normal(0, 1, df_merged.shape[0])


# Generate PI variable which is the product of price_ and e^(income)
df_merged['PI'] = df_merged['price_'] * df_merged['income']
df_merged

/tmp/ipykernel_20075/535779265.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1['brand_'] = df1['brand'].isin(range(1, 10)).astype(int)


,store,week,brand,sales_,count,price_,prom_,Hausman,sales_per_count,share_0,...,pricestore24,pricestore25,pricestore26,pricestore27,pricestore28,pricestore29,pricestore30,cost_,V,PI
0,2,1,1,16,14181,3.29,0.0,3.300641,0.001128,0.993724,...,3.31,3.35,3.37,3.31,3.28,3.29,3.24,2.06,0.496714,34.720061
1,2,2,1,12,13965,3.27,0.0,3.298750,0.000859,0.993913,...,3.37,3.24,3.35,3.29,3.30,3.29,3.36,2.04,-0.138264,34.508997
2,2,3,1,6,13538,3.37,0.0,3.311178,0.000443,0.995199,...,3.31,3.26,3.32,3.29,3.29,3.24,3.31,2.15,0.647689,35.564318
3,2,4,1,12,13735,3.30,0.0,3.308676,0.000874,0.993957,...,3.34,3.31,3.41,3.31,3.28,3.31,3.34,2.07,1.523030,34.825593
4,2,5,1,10,13735,3.34,0.0,3.298289,0.000728,0.995268,...,3.32,3.28,3.26,3.32,3.28,3.31,3.33,2.12,-0.234153,35.247721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38539,123,44,11,21,28564,4.12,0.0,3.982733,0.000735,0.995344,...,4.11,3.63,3.81,4.42,3.53,3.90,3.74,1.85,0.889808,42.576492
38540,123,45,11,8,28938,4.27,0.0,4.011566,0.000276,0.994747,...,4.34,3.96,3.66,4.10,3.93,4.11,3.99,1.94,-3.211219,44.126607
38541,123,46,11,8,25438,4.47,0.0,3.972881,0.000314,0.995007,...,3.91,4.50,3.86,3.69,4.08,3.94,3.75,2.08,-0.429882,46.193427
38542,123,47,11,13,25870,4.47,0.0,4.020889,0.000503,0.993235,...,4.39,4.26,3.99,4.17,3.82,4.23,4.39,2.08,-1.334238,46.193427


In [21]:

########################################## Contraction Mapping ###################################################
#The model that I'm considering is u=beta*prom_+alpha*price_+sigma_i*PI+beta_i*V*brand_+Zeta+epsilon of the U_i into
# two parts (excluding epsilon): delta_i=beta*prom__i+alpha*price__i+sigma_i*PI_i+Zeta_i and mu_i=sigma_b*V_i*brand__i.
# define 0_sigma_i=integral (e^(delta_i+mu_i))/(1+sum_{k}e^(delta_k+mu_k))f(v)dv where f is the density of standard normal
# and V is the vector 'V' from out dataframe. We will need numerical integration of this in the end.  Now the steps are, 
# fix sigma_b at some level and start the procedure: pick 0_delta which is the vector of 0_delta_i's where 0_delta_i for 
# a given individual is its corresponding value of Y variable from the data frame. Then update 0_delta vector to 1_delta 
# vector the following way-> 1_delta_i=0_delta_i+log(sales_per_count_i)-log(0_sigma_i). 
# continue this process until the distance between the VECTORS t_delta and {t+1}_delta converges to 0 (up to 3 decimal points) 




In [22]:


# Calculate the initial delta
delta_0 = df_merged['Y']
df_merged['delta_0'] = delta_0

# Generate mu for each entry
sigma_b = 0.00000001
df_merged['mu'] = sigma_b * df_merged['V'] * df_merged['brand_']

# Calculate sigma_0
sigma_0denom = 1 + np.exp(df_merged['delta_0'] + df_merged['mu']).sum()
sigma_0numer = np.exp(df_merged['delta_0'] + df_merged['mu'])
sigma_0 = sigma_0numer / sigma_0denom
df_merged['sigma_0'] = sigma_0

# Update delta
delta_1 = delta_0 + np.log(df_merged['sales_per_count']) - np.log(sigma_0)
df_merged['delta_1'] = delta_1

# Calculate sigma_1

sigma_1denom = 1 + np.exp(df_merged['delta_1'] + df_merged['mu']).sum()
sigma_1numer = np.exp(df_merged['delta_1'] + df_merged['mu'])
sigma_1 = sigma_1numer / sigma_1denom

df_merged['sigma_1'] = sigma_1


# update delta

delta_2 = delta_1 + np.log(df_merged['sales_per_count']) - np.log(sigma_1)
df_merged['delta_2'] = delta_2

# Calculate sigma_2

sigma_2denom = 1 + np.exp(df_merged['delta_2'] + df_merged['mu']).sum()
sigma_2numer = np.exp(df_merged['delta_2'] + df_merged['mu'])
sigma_2 = sigma_2numer / sigma_2denom

df_merged['sigma_2'] = sigma_2


# Continue iterating this way 5-6 times

for t in range(3, 8):
    # Calculate sigma_t
    sigma_t_denom = 1 + np.exp(df_merged[f'delta_{t-1}'] + df_merged['mu']).sum()
    sigma_t_numer = np.exp(df_merged[f'delta_{t-1}'] + df_merged['mu'])
    sigma_t = sigma_t_numer / sigma_t_denom
    df_merged[f'sigma_{t}'] = sigma_t

    # Update delta_t
    delta_t = df_merged[f'delta_{t-1}'] + np.log(df_merged['sales_per_count']) - np.log(sigma_t)
    df_merged[f'delta_{t}'] = delta_t

# Display the final delta and sigma values
df_merged[[f'delta_{t}' for t in range(8)] + [f'sigma_{t}' for t in range(8)]].head()





,delta_0,delta_1,delta_2,delta_3,delta_4,delta_5,delta_6,delta_7,sigma_0,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,sigma_6,sigma_7
0,-6.780774,-3.802917,-0.873650,2.053055,4.979623,7.906184,10.832744,13.759304,0.000057,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060,0.000060
1,-7.053298,-4.075250,-1.145983,1.780722,4.707290,7.633851,10.560411,13.486971,0.000044,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046
2,-7.716683,-4.737344,-1.808077,1.118628,4.045196,6.971757,9.898317,12.824878,0.000023,0.000024,0.000024,0.000024,0.000024,0.000024,0.000024,0.000024
3,-7.036735,-4.058643,-1.129377,1.797329,4.723897,7.650457,10.577018,13.503578,0.000044,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047,0.000047
4,-7.220374,-4.240965,-1.311698,1.615007,4.541575,7.468136,10.394696,13.321257,0.000037,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039
